In [38]:
import pandas as pd
stock_code = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13',
                       header=0)[0]
stock_code.종목코드=stock_code.종목코드.map('{:06d}'.format)

stock_code=stock_code[['회사명','종목코드']]

stock_code=stock_code.rename(columns={'회사명':'company','종목코드':'code'})
stock_code.to_csv('/Users/YoungWoo/stock/company.csv', index=False, encoding='UTF-8')
#code_df.head()
stock_code.head()


,company,code
0,DSR,155660
1,GS글로벌,001250
2,HDC현대산업개발,294870
3,KG케미칼,001390
4,LG이노텍,011070


In [45]:
import requests
from bs4 import BeautifulSoup

plusUrl = input('회사:').upper()
plusUrl = stock_code[stock_code.company==plusUrl].code.values[0].strip()
#print(plusUrl)

def refine_price(text):
    price=int(text.replace(",",""))
    return price
#print(type(price))

result=[]
for i in range(1,90):
    url='https://finance.naver.com/item/sise_day.nhn?code='+str(plusUrl)+'&page={}'.format(i)
    response=requests.get(url)
    text=response.text
    html=BeautifulSoup(text,'html.parser')
    table0=html.find_all("tr",{"onmouseover":"mouseOver(this)"})
    #print(url)
    
    for tr in table0:
        date= tr.find_all('td')[0].text
        temp=[]  
        
        for idx,td in enumerate(tr.find_all('td')[1:]):
            if idx==1:
                try:
                    #print(td.find()['alt'])
                    temp.append(td.find()['alt'])
                except: 
                    temp.append('')
                
            price=refine_price(td.text)
            #print(price)
            temp.append(price)
        
        #print([date]+temp)
        result.append([date]+temp)
        
#print(result)        

In [46]:
df_temp=pd.DataFrame(result,columns=['date','close','up/down','pastday','open','high','low','volume']) #'날짜','종가','상승/하락','전일대비','시가','고가','저가','거래량'
df_temp

,date,close,up/down,pastday,open,high,low,volume
0,2020.10.05,370500,상승,6000,360500,370500,358000,519982
1,2020.09.29,364500,하락,4500,374500,375000,364000,612054
2,2020.09.28,369000,상승,17500,356500,372000,354000,674705
3,2020.09.25,351500,하락,500,356000,358000,346500,449701
4,2020.09.24,352000,하락,13500,361000,361500,351000,763638
...,...,...,...,...,...,...,...,...
885,2017.02.22,89500,상승,1100,88600,90200,88200,1034489
886,2017.02.21,88400,상승,3600,85700,89100,85700,1573781
887,2017.02.20,84800,상승,100,84400,85300,83600,294768
888,2017.02.17,84700,상승,2300,82400,85300,82400,563691


In [47]:
df_stock=df_temp.replace('','동일')
df_stock

,date,close,up/down,pastday,open,high,low,volume
0,2020.10.05,370500,상승,6000,360500,370500,358000,519982
1,2020.09.29,364500,하락,4500,374500,375000,364000,612054
2,2020.09.28,369000,상승,17500,356500,372000,354000,674705
3,2020.09.25,351500,하락,500,356000,358000,346500,449701
4,2020.09.24,352000,하락,13500,361000,361500,351000,763638
...,...,...,...,...,...,...,...,...
885,2017.02.22,89500,상승,1100,88600,90200,88200,1034489
886,2017.02.21,88400,상승,3600,85700,89100,85700,1573781
887,2017.02.20,84800,상승,100,84400,85300,83600,294768
888,2017.02.17,84700,상승,2300,82400,85300,82400,563691


In [42]:
df_stock['up/down']=df_stock['up/down'].map({'하락':0,'상승':1,'동일':2})
df_stock

,date,close,up/down,pastday,open,high,low,volume
0,2020.10.05,187000,1,8500,183000,187500,181500,2707275
1,2020.09.29,178500,1,3000,178500,180000,177500,1475249
2,2020.09.28,175500,1,5500,173000,176000,172000,1374871
3,2020.09.25,170000,0,1500,172500,174000,166000,2076987
4,2020.09.24,171500,0,8000,173500,176000,171000,2841908
...,...,...,...,...,...,...,...,...
885,2017.02.22,149500,0,1500,150000,151000,147000,423978
886,2017.02.21,151000,1,1000,150000,151500,149000,324845
887,2017.02.20,150000,0,500,149500,151000,149500,239916
888,2017.02.17,150500,2,0,149500,151000,148000,484244


In [48]:
df_stock['date']=pd.to_datetime(df_stock['date'])
df_stock.set_index('date', inplace=True)
df_stock.to_csv('/Users/YoungWoo/stock/kakao.csv',encoding='UTF-8')
df_stock.head()

,close,up/down,pastday,open,high,low,volume
date,,,,,,,
2020-10-05,370500,상승,6000,360500,370500,358000,519982
2020-09-29,364500,하락,4500,374500,375000,364000,612054
2020-09-28,369000,상승,17500,356500,372000,354000,674705
2020-09-25,351500,하락,500,356000,358000,346500,449701
2020-09-24,352000,하락,13500,361000,361500,351000,763638


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df_stock['close'].plot(figsize=(12,6), grid=True);

In [ ]:
# 반응형 차트 그리는 코드
fig = px.line(df, x='date', y='close', title='{}의 종가(close) Time Series'.format(company))

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
단순형 차트
import matplotlib.pyplot as plt
# 필요한 모듈 import 하기 
import plotly
import plotly.graph_objects as go
import plotly.express as px

# %matplotlib inline 은 jupyter notebook 사용자용 - jupyter notebook 내에 그래프가 그려지게 한다.
%matplotlib inline 

plt.figure(figsize=(10,4))
plt.plot(df['date'], df['close'])
plt.xlabel('')
plt.ylabel('close')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.savefig(company + ".png")
plt.show()

In [ ]:
import cx_Oracle #오라클 접속 모듈 로딩
#오라클 서버에 접속 '아이디/비번@호스트:포트번호/sid'
conn=cx_Oracle.connect('hr/hr@localhost:1521/xe')
cursor=conn.cursor() #커서생성
sql='''
select * from stock
'''
cursor.execute(sql) #sql 명령어 실행
rows=cursor.fetchall() #결과셋을 리스트로 저장
print('회사\t코드')
for row in rows:
    print('{}\t{}'.format(row[0],row[1]))
#데이터베이스 관련 리소스 정리
cursor.close()
conn.close()